# INSY 6800 - EDA Project 

Project by Richie Wilbanks and Carly Walker

Project Description: Display EDA skills learned in INSY 6800 course by using a real world data set. The data set explored displays statistics for D1 NCAA Men's College Basketball teams ranging from 2010 to 2025. 

## Initial Data Setup (carly section)

In this section the data is imported from github, cleaned, and formatted into a dataframe so that the team can use the data for exploration. 

In [25]:
!pip install openpyxl
#Install to reach excel files 

In [26]:
#import pandas to handle DataFrames and Path for importing dataset
import pandas as pd
from pathlib import Path

#join current working directory to dataset
Project_Root = Path(".")
Data_Dir = Project_Root / "sports_data"

#import and sort excel files from dataset
excel_files = sorted(Data_Dir.glob("*.xls*"))

In [27]:
#Use function to load excel data for all seasons
def clean_season(path, season_name):

    # returns filename extension (.xls or .xlsx)
    ext = path.suffix.lower()

    # Run loop so function can support different file topes 
    if ext == ".xls":
        tables = pd.read_html(path, header=[0, 1])
        raw = tables[0]
    elif ext == ".xlsx":
        raw = pd.read_excel(path, header=[0, 1], engine="openpyxl")
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    #Flatten column names 
    new_cols = []
    for col in raw.columns:
        # col is a tuple (ex: ("Overall", "W")
        if isinstance(col, tuple):
            top = str(col[0])
            sub = str(col[1])
        else:
            top = ""
            sub = str(col)

        # Cleaning data 
            #if school has no name
        if top.startswith("Unnamed") and sub == "School":
            name = "School"
        elif top == "Overall":
            # Tuples start with "Overall" but we want the sub header (ex: W, L, G)
            name = sub
        elif top == "Totals":
            # Tupes start with "Total" but we want sub header (ex: FG, 3P)
            name = sub
        elif top == "Points" and sub == "Tm.":
            #Make more readable 
            name = "PTS"
        elif top == "Points" and sub == "Opp.":
            #Make more readable 
            name = "Opp PTS"
        else:
            # For Conf., Home, Away, etc., create a unique name
            # that won't match keep_cols (so we ignore them)
            name = f"{top}_{sub}".strip()

        new_cols.append(name)

    raw.columns = new_cols

    #drop any completely empty columns
    raw = raw.dropna(axis=1, how="all")

    #Keep only wanted stats 
    keep_cols = [
        "School",
        "G", "W", "L", "W-L%", "SRS", "SOS",
        "PTS", "Opp PTS",
        "FG", "FGA", "FG%",
        "3P", "3PA", "3P%",
        "FT", "FTA", "FT%",
        "TRB", "AST", "STL", "BLK", "TOV"
    ]

    existing_cols = [col for col in keep_cols if col in raw.columns]
    missing_cols = [col for col in keep_cols if col not in raw.columns]

    #warning if column is missing 
    if missing_cols:
        print(f"For {season_name}, missing columns (skipped): {missing_cols}")

    df = raw[existing_cols].copy()

    #Remove duplicates 
    df = df.loc[:, ~df.columns.duplicated()]

    # Clean school names 
    if "School" in df.columns:
        df["School"] = (
            df["School"]
            .astype(str)
            .str.replace(r"\s*NCAA$", "", regex=True)
            .str.replace(r"\s*\(.*?\)", "", regex=True)
            .str.strip()
        )

    # Convert all numeric columns 
    for col in df.columns:
        if col != "School":
            #Ensures we are working with a series and not a data frame
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Add a season column
    df["Season"] = season_name

    return df

In [28]:
#Creates a dictionary with the season name (ex: 2010-2011) as key name and values as the dataframe from above 
season_dfs = {}

#Loop through excel files and strip so season name is the years 
for file in excel_files:
    name = file.stem
    season_name = name.split("_")[-1]  # ex: 2024-2025

    df_season = clean_season(file, season_name) # use fuction from above 
    season_dfs[season_name] = df_season

    #confirm the season was loaded 
    print("Loaded season:", season_name, "Shape:", df_season.shape)


Loaded season: 2010-2011 Shape: (345, 24)
Loaded season: 2011-2012 Shape: (344, 24)
Loaded season: 2012-2013 Shape: (347, 24)
Loaded season: 2013-2014 Shape: (351, 24)
Loaded season: 2014-2015 Shape: (351, 24)
Loaded season: 2015-2016 Shape: (351, 24)
Loaded season: 2016-2017 Shape: (351, 24)
Loaded season: 2017-2018 Shape: (351, 24)
Loaded season: 2018-2019 Shape: (353, 24)
Loaded season: 2019-2020 Shape: (353, 24)
Loaded season: 2020-2021 Shape: (347, 24)
Loaded season: 2021-2022 Shape: (358, 24)
Loaded season: 2022-2023 Shape: (363, 24)
Loaded season: 2023-2024 Shape: (363, 24)
Loaded season: 2024-2025 Shape: (364, 24)


In [30]:
#concatenate all cleaned seasons - create one data frame with all seasons data 
df_all = pd.concat(season_dfs.values(), ignore_index=True)
# remove rows with missing values
df_all = df_all.dropna().reset_index(drop=True)

## Question 1: What factors correlate with winning? (carly section) 

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

## Question 2: Are stats shifting over time? (carly section)

## Question 3: Do certain teams have statisical "signatures"? (carly Section)

## Question 4: Outliers or breakout seasons? (carly)

## Question 5: Richie

## Question 6: Richie

## Question 7: Richie

## Question 8: Richie 

## Takeaways/Conclusion: Richie 